# Homework 5 #

Import statements, load NMC graphite parameters

In [4]:
import pybamm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%load_ext autoreload


In [ ]:
%autoreload 2

from parameters.NMC_graphite_parameters import get_parameter_values

parameter_values = get_parameter_values()
parameter_values

Display mission data for chosen aircraft Beta Alia, assign max takeoff and structural mass

In [ ]:
import pandas as pd

filenames = ["Lilium_Jet", "KH_Heaviside", "Joby_5seater", "Beta_Alia", "Archer_Maker"]
filename = filenames[3]
df = pd.read_csv("powerprofiles/" + filename + ".csv")
df

In [7]:
mtom_beta_alia = 2730 #kg
structure_weight_beta_alia = 0.4*mtom_beta_alia

Part 1

In [ ]:
# (i) Cell nominal voltage
nom_V = 3.63 # V

# (ii) Cell nominal capacity
nom_E = 18.2 #Wh
nom_C = nom_E/nom_V #Ah

print(f"Nominal Voltage (V): {nom_V}")
print(f"Nominal Capacity (Ah): {nom_C}")

# (iii) Max discharge rate (in A)
max_D_current = 7275e-3 # A
max_D_rate = 1.5 # C
print(f"Max Discharge Rate (A): {max_D_rate}")
### Max discharge rate is 1.5C, not the current 

weight = 68e-3 # kg
max_V = 4.2
spec_P = max_D_current*max_V/weight

# (iv) Cell specific energy
spec_E = spec_P*2/3/weight
print(f"Specific Energy (Wh/kg): {spec_E}")

# (v) Cell specific power
print(f"Specific Power (W/kg): {spec_P}")

# (vi) Voltage limits of operation
min_V = 2.5 
print(f"Minimum Voltage (V): {min_V}")
print(f"Maximum Voltage (V): {max_V}")

In [20]:
from typing import List

In [ ]:
List[float]

Part 2

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# load the file and clean the data
file_path_neg = 'C:/Users/hueyf/UMich/CMU_energy_storage_HW5/parameters/graphite_LGM50_ocp_Chen2020.csv'
file_path_pos = 'C:/Users/hueyf/UMich/CMU_energy_storage_HW5/parameters/nmc_LGM50_ocp_Chen2020.csv'

files = [file_path_neg, file_path_pos]
rk_polynomials = []

for file_path in files:
    data = pd.read_csv(file_path, comment='#', header=None, names=['sto', 'ocp'])
    data_cleaned = data.dropna()
    data_cleaned = data_cleaned.apply(pd.to_numeric, errors='coerce').dropna()

    # RK model with an offset
    def rk_model_with_offset(x, offset, *coefficients):
        n = len(coefficients)
        rk_sum = sum(coefficients[i] * x*(1-x) * (1 - 2*x)**i for i in range(n))
        return offset + rk_sum

    # Prepare the data for fitting
    x_data = data_cleaned['sto'].values  # SOC values
    y_data = data_cleaned['ocp'].values  # OCP values

    # Function to calculate residuals (Sum of Squared Residuals, SSR)
    def calculate_residuals(x: float, y, model, coefficients):
        """
        ArithmeticError

        Parameters
        x: float

        Returns

        """
        y_pred = model(x, *coefficients)
        return np.sum((y - y_pred)**2)

    # Start with a reduced range for polynomial orders to avoid overfitting
    best_fit_reduced = None
    best_residuals_reduced = float('inf')
    best_order_reduced = 0

    # Iterate over polynomial orders 
    for order in range(5, 11):
        initial_guess = [1.0] * (order + 1)  # Include an initial guess for the offset
        try:
            popt, pcov = curve_fit(rk_model_with_offset, x_data, y_data, p0=initial_guess)
            residuals = calculate_residuals(x_data, y_data, rk_model_with_offset, popt)
            if residuals < best_residuals_reduced:
                best_residuals_reduced = residuals
                best_fit_reduced = popt
                best_order_reduced = order
        except RuntimeError:
            continue  # Skip orders that fail to converge

    # Generate a smooth curve for the reduced RK fit
    x_fit = np.linspace(min(x_data), max(x_data), 500)
    y_fit = rk_model_with_offset(x_fit, *best_fit_reduced)

    # Plot the data and the reduced RK fit curve
    plt.figure(figsize=(8, 5))
    plt.scatter(x_data, y_data, label="Data", color="blue", alpha=0.7)
    plt.plot(x_fit, y_fit, label=f"Reduced RK Fit (Order {best_order_reduced})", color="red")
    plt.xlabel("State of Charge (SOC)")
    plt.ylabel("Open-Circuit Potential (OCP)")
    plt.title(f"Reduced RK Polynomial Fit to OCP vs. SOC (Order {best_order_reduced})")
    plt.legend()
    plt.grid(True)
    plt.show()

    rk_polynomials.append(best_fit_reduced)

print(rk_polynomials)

Part 3

In [ ]:
import pybamm

# Define RK OCP functions
# Negative electrode OCP based on RK polynomial.
def graphite_LGM50_ocp_Chen2020(sto):
    coeffs = rk_polynomials[0]
    return sum(c * sto * (1 - sto)**i for i, c in enumerate(coeffs[1:]))+coeffs[0]

# Positive electrode OCP based on RK polynomial
def nmc_LGM50_ocp_Chen2020(sto):
    coeffs = rk_polynomials[1]
    return sum(c * sto * (1 - sto)**i for i, c in enumerate(coeffs[1:]))+coeffs[0]

initial_soc = 0.5  # Default SOC for initial conditions
initial_voltage = nmc_LGM50_ocp_Chen2020(initial_soc) - graphite_LGM50_ocp_Chen2020(initial_soc)
print(f"Initial Cell Voltage: {initial_voltage} V")


# Create PyBaMM model and parameter set
model = pybamm.lithium_ion.DFN()  # Doyle-Fuller-Newman model
parameter_values = get_parameter_values()
parameter_values.update({
    "Negative electrode OCP [V]" : graphite_LGM50_ocp_Chen2020,
    "Positive electrode OCP [V]" : nmc_LGM50_ocp_Chen2020}, 
    check_already_exists=False)

parameter_values.update({
    # cathode-anode OCV for STO = 0, 1 respectively 
    "Open-circuit voltage at 100% SOC [V]" : parameter_values['Upper voltage cut-off [V]'],
    # cathode-anode OCV for STO = 1, 0 respectively 
    "Open-circuit voltage at 0% SOC [V]" : parameter_values['Lower voltage cut-off [V]']}, 
    check_already_exists=False)

# Create and solve the simulation
# Set up the experiment for 1C constant current discharge
experiment = pybamm.Experiment([
    "Discharge at 1C until 2.5V"
])

simulation = pybamm.Simulation(model, parameter_values=parameter_values, experiment=experiment)
solution = simulation.solve()

# Plot the results
simulation.plot()


In [ ]:
pybamm.__version__

Part 4

In [ ]:
# Calculate total energy, peak power, and required cells for EVTOL.
def calculate_battery_metrics(file_path, cell_energy_wh, cell_power_w):
    # Load data
    data = pd.read_csv(file_path)
    time = data['Times']  # Assuming time in seconds
    power = data['PowerkW']  # Assuming power in kW

    # Calculate total energy required (kWh)
    total_energy_kwh = np.trapz(power, time) / 3600  # Convert Ws to kWh

    # Calculate peak power (kW)
    peak_power_kw = max(power)

    # Calculate required cells for energy
    total_cells_energy = total_energy_kwh * 1000 / cell_energy_wh

    # Calculate required cells for peak power
    total_cells_power = peak_power_kw * 1000 / cell_power_w

    return total_energy_kwh, peak_power_kw, total_cells_energy, total_cells_power

# Parameters for LGM50 cell
cell_energy_wh = nom_E  # Energy per cell in Wh
cell_power_w = spec_P*weight # Maximum power per cell in W

# plotting function 
def plot_power_vs_time(evtol, file_path):
    # Load data
    data = pd.read_csv(file_path)
    time = data['Times']  # Assuming time in seconds
    power = data['PowerkW']  # Assuming power in kW

    # Plot power vs. time
    plt.plot(time, power, label=evtol)

    plt.xlabel("Time (s)")
    plt.ylabel("Power (kW)")
    plt.title("Power vs Time for EVTOL Designs")
    plt.legend()
    plt.grid(True)
    plt.show()

# EVTOL file paths
evtol_files = {
    "Archer Maker": "C:/Users/hueyf/UMich/CMU_energy_storage_HW5/powerprofiles/Archer_Maker.csv",
    "Beta Alia": "C:/Users/hueyf/UMich/CMU_energy_storage_HW5/powerprofiles/Beta_Alia.csv",
    "Joby 5-seater": "C:/Users/hueyf/UMich/CMU_energy_storage_HW5/powerprofiles/Joby_5seater.csv",
    "KH Heaviside": "C:/Users/hueyf/UMich/CMU_energy_storage_HW5/powerprofiles/KH_Heaviside.csv",
    "Lilium Jet": "C:/Users/hueyf/UMich/CMU_energy_storage_HW5/powerprofiles/Lilium_Jet.csv"
}

beta_alia_cells = 0
for evtol, file_path in evtol_files.items():

    total_energy_kwh, peak_power_kw, total_cells_energy, total_cells_power = calculate_battery_metrics(file_path, cell_energy_wh, cell_power_w)
    if evtol == "Beta Alia":   
        beta_alia_cells = total_cells_energy
    print(f"EVTOL: {evtol}")
    # Plot power vs time
    plot_power_vs_time(evtol, file_path)
    print(f"Total Energy Required: {total_energy_kwh:.2f} kWh")
    print(f"Peak Power Required: {peak_power_kw:.2f} kW")
    print(f"Cells Required for Energy: {total_cells_energy:.0f}")
    print(f"Cells Required for Peak Power: {total_cells_power:.0f}")
    print("_________________________________________________________________________________________________")

Part 5

In [ ]:
import math
print(math.ceil(beta_alia_cells))

In [ ]:
# Create a PyBaMM experiment for an EVTOL mission using constant-power segments.
def create_evtol_experiment(file_path, total_cells):
    # Load EVTOL power profile
    data = pd.read_csv(file_path)
    time = data['Times']  # Assuming time in seconds
    power = data['PowerkW']  # Assuming power in kW

    # Calculate power per cell
    power_per_cell = power / total_cells

    # Break the mission into constant-power segments
    mission_segments = []
    for i in range(len(time) - 1):
        duration_minutes = (time[i + 1] - time[i]) / 60  # Convert duration to minutes
        power_kw = power_per_cell[i]*1e3
        segment = f"Discharge at {power_kw:.2f} W for {duration_minutes:.2f} minutes"
        mission_segments.append(segment)

    # Create PyBaMM experiment
    experiment = pybamm.Experiment(mission_segments)
    return experiment


experiment = create_evtol_experiment("C:/Users/hueyf/UMich/CMU_energy_storage_HW5/powerprofiles/Beta_Alia.csv", math.ceil(beta_alia_cells))

print(experiment)
# Create simulation with the experiment
simulation = pybamm.Simulation(model, parameter_values=parameter_values, experiment=experiment)
solution = simulation.solve()

# Plot the results
simulation.plot()


Part 6

In [ ]:
# Function to adjust the number of cells to complete the EVTOL mission
def adjust_cells_for_mission(file_path, cell_energy_wh): 
    data = pd.read_csv(file_path)
    time = data['Times']  # Assuming time in seconds
    power = data['PowerkW']  # Assuming power in kW

    # Calculate total energy required (kWh)
    total_energy_kwh = np.trapz(power, time) / 3600  # Convert Ws to kWh

    # Start with an estimated number of cells and adjust
    estimated_cells = int(np.ceil(total_energy_kwh * 1000 / cell_energy_wh))
    while True:
        power_per_cell = power / estimated_cells
        if all(power_per_cell * 1000 <= cell_energy_wh):  # Check if energy per cell is feasible
            break
        estimated_cells += 1

    return estimated_cells

# Create a PyBaMM experiment for an EVTOL mission using constant-power segments.
def create_evtol_experiment(file_path, total_cells):
    # Load EVTOL power profile
    data = pd.read_csv(file_path)
    time = data['Times']  # Assuming time in seconds
    power = data['PowerkW']  # Assuming power in kW

    # Calculate power per cell
    power_per_cell = power / total_cells

    # Break the mission into constant-power segments
    mission_segments = []
    for i in range(len(time) - 1):
        duration_minutes = (time[i + 1] - time[i]) / 60  # Convert duration to minutes
        power_kw = power_per_cell[i]*1e3
        segment = f"Discharge at {power_kw:.2f} W for {duration_minutes:.2f} minutes"
        mission_segments.append(segment)

    # Create PyBaMM experiment
    experiment = pybamm.Experiment(mission_segments)
    return experiment

beta_alia_cells = adjust_cells_for_mission("C:/Users/hueyf/UMich/CMU_energy_storage_HW5/powerprofiles/Beta_Alia.csv", cell_energy_wh)
experiment = create_evtol_experiment("C:/Users/hueyf/UMich/CMU_energy_storage_HW5/powerprofiles/Beta_Alia.csv", math.ceil(beta_alia_cells))

print(experiment)
# Create simulation with the experiment
simulation = pybamm.Simulation(model, parameter_values=parameter_values, experiment=experiment)
solution = simulation.solve()

# Plot the results
simulation.plot()

Part 7

Part 8

Part 9

Part 10

Part 11